In [2]:
import requests
import pandas as pd

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API key
API_KEY = os.getenv("API_KEY")

if not API_KEY:
    raise ValueError("API_KEY is not set in the environment variables.")

# Example usage
print(f"My API Key is: {API_KEY}")
base_url = 'https://api.the-odds-api.com/v4/sports/basketball_ncaab/odds'
regions = 'us'
odds_format = 'american'

# Step 1: Fetch all games to get each game's event_id and the full game total
initial_api_url = f'{base_url}?apiKey={API_KEY}&regions={regions}&markets=totals&oddsFormat={odds_format}'
response = requests.get(initial_api_url)

# Check if the response is successful
if response.status_code == 200:
    events = response.json()
    
    # Initialize a list to store parsed data
    all_parsed_data = []
    
    # Loop through each event to get the event_id and full game total
    for event in events:
        event_id = event['id']
        date = event['commence_time']
        home_team = event['home_team']
        away_team = event['away_team']
        
        # Initialize full game total from the main response
        full_game_total = None

        # Find the full game total from the main endpoint response
        fanduel_data = next((bookmaker for bookmaker in event['bookmakers'] if bookmaker['key'] == 'fanduel'), None)
        
        if fanduel_data:
            for market in fanduel_data['markets']:
                if market['key'] == 'totals':
                    for outcome in market['outcomes']:
                        if outcome['name'] == "Over":
                            full_game_total = outcome['point']
        
        # Step 2: For each event_id, fetch additional markets (team_totals, team_totals_h1)
        event_url = f'https://api.the-odds-api.com/v4/sports/basketball_ncaab/events/{event_id}/odds?apiKey={API_KEY}&regions={regions}&markets=team_totals,team_totals_h1,totals,spreads&oddsFormat={odds_format}'
        event_response = requests.get(event_url)
        
        if event_response.status_code == 200:
            event_data = event_response.json()
            
            # Initialize variables for additional totals
            first_half_total = None
            home_team_total = None
            away_team_total = None
            home_team_total_1h = None
            away_team_total_1h = None
            home_spread = None
            
            # Look for FanDuel data in the event-specific response
            fanduel_data_detailed = next((bookmaker for bookmaker in event_data['bookmakers'] if bookmaker['key'] == 'fanduel'), None)
            
            if fanduel_data_detailed:
                # Check each market for FanDuel
                for market in fanduel_data_detailed['markets']:
                    # First half total (totals_h1)
                    if market['key'] == 'totals_h1':
                        for outcome in market['outcomes']:
                            if outcome['name'] == "Over":
                                first_half_total = outcome['point']

                    # Team totals for the full game
                    elif market['key'] == 'team_totals':
                        for outcome in market['outcomes']:
                            if outcome['name'] == "Over" and outcome['description'] == home_team:
                                home_team_total = outcome['point']
                            elif outcome['name'] == "Over" and outcome['description'] == away_team:
                                away_team_total = outcome['point']

                    # Team totals for the first half (team_totals_h1)
                    elif market['key'] == 'team_totals_h1':
                        for outcome in market['outcomes']:
                            if outcome['name'] == "Over" and outcome['description'] == home_team:
                                home_team_total_1h = outcome['point']
                            elif outcome['name'] == "Over" and outcome['description'] == away_team:
                                away_team_total_1h = outcome['point']
                    elif market['key'] == 'spreads':
                        for outcome in market['outcomes']:
                            if outcome['name'] == home_team:
                                home_spread = outcome['point']
                            


            all_parsed_data.append({
                'date': date,
                'home_team': home_team,
                'away_team': away_team,
                'full_game_total': full_game_total,
                # 'first_half_total': first_half_total,
                'home_team_total': home_team_total,
                'away_team_total': away_team_total,
                'home_spread': home_spread
                # 'home_team_total_1h': home_team_total_1h,
                # 'away_team_total_1h': away_team_total_1h
            })

    # Step 3: Convert the parsed data into a DataFrame
    df = pd.DataFrame(all_parsed_data)
    print(df)

else:
    print(f"Error: {response.status_code} - {response.text}")

My API Key is: 023e59418c07e01bccdb1786fc283374
                    date                     home_team  \
0   2024-11-20T23:00:00Z          The Citadel Bulldogs   
1   2024-11-20T23:30:00Z             Xavier Musketeers   
2   2024-11-20T23:30:00Z         Stony Brook Seawolves   
3   2024-11-21T00:00:00Z              Kennesaw St Owls   
4   2024-11-21T00:00:00Z                  Army Knights   
5   2024-11-21T00:00:00Z            Morehead St Eagles   
6   2024-11-21T00:00:00Z          Ball State Cardinals   
7   2024-11-21T00:00:00Z                    LIU Sharks   
8   2024-11-21T00:00:00Z         George Mason Patriots   
9   2024-11-21T00:00:00Z                 Dayton Flyers   
10  2024-11-21T00:00:00Z     Northern Illinois Huskies   
11  2024-11-21T00:00:00Z      Penn State Nittany Lions   
12  2024-11-21T00:00:00Z    West Virginia Mountaineers   
13  2024-11-21T00:00:00Z          Virginia Tech Hokies   
14  2024-11-21T00:00:00Z             Rhode Island Rams   
15  2024-11-21T00:00:00Z

In [2]:
df.to_csv("/Users/nickdimmitt/Desktop/lumber/ncaab/daily-odds.csv")
df.to_csv("predict/daily-odds.csv")